## Removed by AdapterRemoval and kept by fastp

This is a sequence that Fastp kept and AdapterRemoval removed

In [1]:
seq = 'ANCTCGCCTATTGCCTCGGTGAAAGCGAATTATTGGAGAGAGCACGGCCTGACGAGAGACGTTTGGCGCCGTTTTATTCGTTCACCGTGACCACGATCTCGCGTATTGCCTCGGTGAAAGCGAATTATTGGAGAGTGCACGGC'


In [3]:
import pandas as pd

nextera = pd.read_csv('../pre_assembly_qc/nextera_pe_adapters.csv', header=None)
nextera

,0,1
0,AGATGTGTATAAGAGACAG,AGATGTGTATAAGAGACAG
1,TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG,GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG
2,CTGTCTCTTATACACATCTGACGCTGCCGACGA,CTGTCTCTTATACACATCTCCGAGCCCACGAGAC


In [7]:
def hamming(seq1, seq2):
    return sum(s1 != s2 for s1, s2 in zip(seq1, seq2))

hamming("asdf", "asdf")

0

In [18]:
def lowest_hamming(query, subject, min_overlap=8):
    """finds position in subject with lowest hamming distance to query
    
    assumes: len(subject) > len(query)
    """
    
    query_len = len(query)
    min_hamming = query_len
    argmin_hamming = None
    
    for i in range(len(subject) - min_overlap + 1):
        substring = subject[i:(i+query_len)]
        if len(substring) < query_len:
            q = query[:len(substring)]
        else:
            q = query
            
        h = hamming(q, substring)
        if h < min_hamming:
            argmin_hamming = i
            min_hamming = h
    return i, min_hamming

In [19]:
nextera.applymap(lambda x: lowest_hamming(x, seq))

,0,1
0,"(135, 6)","(135, 6)"
1,"(135, 3)","(135, 5)"
2,"(135, 7)","(135, 7)"


In [20]:
seq[135:]

'TGCACGGC'

This looks like the beginning of the `TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG` sequence

In [11]:
hamming(seq[124:], nextera.iloc[0, 0])

15

In [12]:
len(seq)

143

In [42]:
seq[140:150]

'GCG'

## Kept by AdapterRemoval and removed by fastp

fastp looks at overlap between R1 and R2 and infers adapters from there

In [22]:
r1_seq = 'TNTCTCGTCAGGCCGTGCTCGCTCCAATAATTCGCTTTCACCGAGGCAATAGCCGAGATCGTGGTCACGTGGAACGAATAAAACGGCGCCAATCTACTCTCGTCAGGCCGTGCTCGCTCCAATAATTCGCTTTCTCCGAGGCG'

In [23]:
r2_seq = 'NCCATTGCCTCGGTGAAAGCGAATTATTGGAGCGAGCACGGCCTGACGAGAGACTTTAGGCGAGGGTTTAGTCGTTCTCCGTGACCACGAGATCGGCTATTTCCTAGGTGAAGGCGGAGTATTGGAGCGAGCACGGCGTGACG'

In [24]:
lowest_hamming(r1_seq, r2_seq)

(135, 7)

In [26]:
r2_seq[135:]

'GCGTGACG'

In [31]:
def reverse_complement(sequence):
    complements = {"A": "T", "G": "C", "T": "A", "C": "G", "N": "N"}
    return ''.join(reversed([complements[s] for s in sequence]))
r2_rc = reverse_complement(r2_seq)
r2_rc

'CGTCACGCCGTGCTCGCTCCAATACTCCGCCTTCACCTAGGAAATAGCCGATCTCGTGGTCACGGAGAACGACTAAACCCTCGCCTAAAGTCTCTCGTCAGGCCGTGCTCGCTCCAATAATTCGCTTTCACCGAGGCAATGGN'

In [32]:
lowest_hamming(r1_seq, r2_rc)

(135, 3)

In [33]:
lowest_hamming(r2_rc, r1_seq)


(135, 5)

In [34]:

r1_seq[135:]

'CCGAGGCG'

In [35]:
hamming(r1_seq[135:], r2_rc[:8])

7

In [ ]:
r2_rc[]

I don't understand why fastp removed this since it doesn't match any adapters.. so I'd rather just use fastp for poly-G removal

## These lines are different and I cna't tell why

In [36]:
adapteremoval = 'ANCCTGCTAAGGCCGGCACCAAGAAAGACCGACGCCGAGGGTCCCATCACTACCTACGCCGCTTTCGGACGCTGCAACGTGGCCACTCCAATAACACCAACTGCAGCCGCCTCTCCGACACCTCACACGACCCCGACC'
fastp = 'ANCCTGCTAAGGCCGGCACCAAGAAAGACCGACGCCGAGGGTCCCATCACTACCTACGCCGCTTTCGGACGCTGCAACGTGGCCACTCCAATAACACCAACTGCAGCCGCCTCTCCGACACCTCACACGACCCCGACCCTGTC'

In [37]:
adapteremoval == fastp

False

In [38]:
''.join('.' if a == b else a for a, b in zip(adapteremoval, fastp))

'..........................................................................................................................................'

In [39]:
len(adapteremoval)

138

In [40]:
len(fastp)

143

In [41]:
fastp[138:]

'CTGTC'

But that's definitely an adapter ... `CTGTCTCTTATACACATCTGACGCTGCCGACGA` is the start of the sequence